In [9]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [10]:
import logging

import duckdb
import holoviews as hv
import hvplot.polars  # noqa
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.data.quality_checks import check_sample_rate
from src.features.scaling import scale_min_max
from src.features.transforming import map_trials
from src.log_config import configure_logging
from src.plots.plot_modality import plot_modality_over_trials
from src.plots.utils import prepare_multiline_hvplot

configure_logging(
    stream_level=logging.DEBUG, ignore_libs=("Comm", "bokeh", "tornado", "matplotlib")
)
logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [12]:
plot_modality_over_trials("ppg", signals=["ppg_raw", "ppg_rate", "ppg_quality"])

BokehModel(combine_events=True, render_bundle={'docs_json': {'acbeeef7-49cc-48ba-b0c0-3c984905102c': {'version…

In [4]:
db = DatabaseManager()

In [5]:
with db:
    df = db.get_table("Feature_PPG")
df

trial_id,trial_number,participant_id,rownumber,timestamp,samplenumber,ppg_raw,ppg_heartrate,ppg_ibi,ppg_clean,ppg_rate,ppg_quality,ppg_peaks
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,1,1,37664.142857,294200.0,57896.142857,1416.012559,-1.0,-1.0,-25.366688,65.896546,0.975548,0.0
1,1,1,37676.5,294300.0,57908.5,1376.495726,-1.0,-1.0,-61.841532,65.896546,0.975548,0.0
1,1,1,37690.875,294400.0,57922.875,1466.941392,-1.0,-1.0,27.796478,65.896546,0.975548,0.0
1,1,1,37704.25,294500.0,57936.25,1522.710623,-1.0,-1.0,88.741595,65.783256,0.974676,0.083333
1,1,1,37714.8,294600.0,57946.8,1516.483516,-1.0,-1.0,77.180253,65.34652,0.971103,0.0
1,1,1,37727.642857,294700.0,57959.642857,1457.613815,-1.0,-1.0,30.559659,64.893857,0.96658,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…
332,12,28,355422.0,2.7766e6,467019.0,1384.102564,73.0,-1.0,3.861983,65.934066,0.987849,0.0
332,12,28,355433.555556,2.7767e6,467030.555556,1343.996744,73.0,-1.0,-47.397796,65.934066,0.987849,0.0


In [9]:
df.hvplot(x="timestamp", y=["ppg_rate", "ppg_quality"], groupby="trial_id", kind="line")

BokehModel(combine_events=True, render_bundle={'docs_json': {'1995cf83-eeaa-4a48-92ed-0ca9cb0e6df6': {'version…